In [65]:
from experiments.evaluation_result_container import EvaluationResultContainer, EvaluationResult
import json
from visualization import mk_fig
import re
from experiments.evaluation_statistics import aggregate_by_exp_type
from visualization import prepare_scalars_for_plot
from experiments.metric import MetricTemplate,Flags
import matplotlib.pyplot as plt

Cross Exp scalar visual

In [23]:
with open(f"saved_data/eval_results/merged/E1.json", "r") as f:
    res_d = json.load(f)
    E1 = EvaluationResultContainer.from_dict(res_d)

In [26]:
E1_1 = aggregate_by_exp_type(E1_1)

Experiment Results Container:
Experiment: E1_1_rep_Y_epoch_25[Y] Results, Type: E_1_1
Experiment: E_1_2_1Y_epoch_25[Y] Results, Type: E_1_2_1
Experiment: E_1_2_2Y_epoch_25[Y] Results, Type: E_1_2_2
Experiment: E_1_2_4Y_epoch_25[Y] Results, Type: E_1_2_4
Experiment: E_1_2_8Y_epoch_25[Y] Results, Type: E_1_2_8
Experiment: E_1_2_16Y_epoch_25[Y] Results, Type: E_1_2_16
Experiment: E_1_2_32Y_epoch_25[Y] Results, Type: E_1_2_32
Experiment: E_1_2_64Y_epoch_25[Y] Results, Type: E_1_2_64